## Load libraries

In [8]:
# Libraries to work cross-platform
import os

# Libraries to work with dataset
import numpy as np
import pandas as pd
import ast                      # convert string to list after importing csv data
import pickle

# Libraries to cluster data
from hdbscan import HDBSCAN

# Libraries to visualize data
import matplotlib.pyplot as plt
import seaborn as sns
# import matplotlib.cm as cm
from cluster_visualization_helper import (
    visualize_cluster)  # user-defined functions

# Libraries for evaluation
from sklearn import metrics

# Libraries for monitoring operation process
from datetime import datetime
from tqdm import tqdm
from joblib import Memory

## Configurate and declare global variables

In [9]:
os_name = os.name

if os_name == 'nt':
    """Windows platform"""
    BASE_DIR = "E:/THIENDHB_GOOGLEDRIVE/MASTER TILBURG/THESIS/"

elif os_name == 'posix':
    """Linux platform"""
    BASE_DIR = "/media/pinkalinux/WORK/THIENDHB_GOOGLEDRIVE/MASTER TILBURG/THESIS/"

INPUT_DIR = BASE_DIR + "DATASET/INPUT/"
OUTPUT_DIR = BASE_DIR + "DATASET/OUTPUT/"
RESULT_DIR = BASE_DIR + "RESULTS/"

SEED = 6886
%matplotlib inline

## Import data

In [10]:
skill_embeddings = np.load(OUTPUT_DIR + "skill_feat_halfsize_embeddings.npy")
skill_embeddings.shape

(257205, 150)

In [11]:
skill_docs = pd.read_csv(
    OUTPUT_DIR + "skill_tokens_long_df.csv",
    converters={
        "skill_token": ast.literal_eval,
    },
    dtype={
        "key_id": int,
        "skill_id": int,
        "job_id": int,
        "column_type": str,
        "skill": str,
        "skill_lemma": str
    }
)
skill_docs.shape

(257205, 9)

In [12]:
skill_docs

,row_id,key_id,job_id,skill_id,column_id,column_type,skill,skill_lemma,skill_token
0,10101,101,1,1,1,job_description,ameria investment consulting company,ameria investment consult company,"[ameria, investment, consult, company]"
1,10201,102,1,2,1,job_description,requires high level,require high level,"[require, high, level]"
2,10301,103,1,3,1,job_description,provides highly responsible,provide highly responsible,"[provide, highly, responsible]"
3,10401,104,1,4,1,job_description,complex staff assistance,complex staff assistance,"[complex, staff, assistance]"
4,10501,105,1,5,1,job_description,chief financial officer,chief financial officer,"[chief, financial, officer]"
...,...,...,...,...,...,...,...,...,...
257200,190010103,1900101,19001,1,3,job_qualification,ra financial system,ra financial system,"[ra, financial, system]"
257201,190010203,1900102,19001,2,3,job_qualification,higher legal education,high legal education,"[high, legal, education]"
257202,190010303,1900103,19001,3,3,job_qualification,high pressure environment,high pressure environment,"[high, pressure, environment]"
257203,190010403,1900104,19001,4,3,job_qualification,professional work experience,professional work experience,"[professional, work, experience]"


## Clustering data

### HDBSCAN

In [13]:
def hdbscan_clusterer(X, min_samples, min_cluster_size, memory):
    """Generate clusters using HDBSCAN method
    Hierarchical Density-Based Spatial Clustering of Applications with Noise

    Args:
        X:                 Matrix of features
                             (n_samples, n_features)
        min_samples:       The number of samples in a neighborhood for a point
                             to be considered as a core point
                             (int, default=None)
        min_cluster_size:  The minimum size of clusters
                             (int, default=5)

    Returns:
        Trained clustering model based on X
    """
    clusterer = HDBSCAN(
        min_samples=min_samples,
        min_cluster_size=min_cluster_size,
        algorithm='boruvka_kdtree',
        gen_min_span_tree=True,
        core_dist_n_jobs=8,
        memory=memory,
    )
    clusterer.fit(X)
    return clusterer

In [14]:
# Define search space for tuning hyperparameters
X = skill_embeddings
mem = Memory(location=OUTPUT_DIR + "/cachedir")
tuning_result = {
    "n_clusters": [],
    "n_noises": [],
    "min_samples": [],
    "min_cluster_size": [],
    "duration": [],
}
model_list = []
label_list = []
outlier_list = []
exemplar_list = []
min_samples_list = [3, 5, 10, 25, 50]
min_cluster_size_list = [2, 3, 4, 5, 10, 25, 50]
len(min_samples_list) * len(min_cluster_size_list)

35

In [ ]:
# Tuning hyperparameter
start_loop_time = datetime.now()
print("Start loop", start_loop_time.strftime("%Y-%m-%d %H:%M:%S.%f"))

for min_samples in tqdm(
    iterable=min_samples_list,
    desc="Tuning HDBSCAN Clustering",
    total=len(min_samples_list) * len(min_cluster_size_list),
):
    for min_cluster_size in min_cluster_size_list:

        # Train model
        starttime = datetime.now()
        print("Start", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))
        print("min_samples =", min_samples)
        print("min_cluster_size =", min_cluster_size)

        clusterer = hdbscan_clusterer(X, min_samples, min_cluster_size, mem)

        endtime = datetime.now()
        print("End", endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
        print("Duration", endtime - starttime)

        # Save model
        pickle.dump(
            clusterer,
            open(
                OUTPUT_DIR
                + "hdbscan/"
                + "skill_hdbscan_model_min_samples-"
                + str(min_samples)
                + "_min_cluster_size-"
                + str(min_cluster_size)
                + ".pkl",
                "wb",
            ),
        )

        model_list.append(clusterer)
        labels = clusterer.labels_
        label_list.append(labels)
        outlier_list.append(clusterer.outlier_scores_)
        exemplar_list.append(clusterer.exemplars_)
        tuning_result["duration"].append(round((endtime - starttime).seconds / 60, 4))
        tuning_result["min_samples"].append(min_samples)
        tuning_result["min_cluster_size"].append(min_cluster_size)
        tuning_result["n_noises"].append(np.sum(np.array(labels) == -1, axis=0))
        tuning_result["n_clusters"].append(np.sum(np.unique(labels) != -1, axis=0))

#         tqdm_bar.update(1)

# tqdm_bar.close()
end_loop_time = datetime.now()
print("End loop", end_loop_time.strftime("%Y-%m-%d %H:%M:%S.%f"))
print("Duration", end_loop_time - start_loop_time)

Tuning HDBSCAN Clustering:   0%|                                                                | 0/35 [00:00<?, ?it/s]

Start loop 2021-05-20 01:12:08.814119
Start 2021-05-20 01:12:08.853176
min_samples = 3
min_cluster_size = 2
________________________________________________________________________________
[Memory] Calling hdbscan.hdbscan_._hdbscan_boruvka_kdtree...
_hdbscan_boruvka_kdtree(array([[-8.807369e-08, ..., -4.767032e-02],
       ...,
       [-9.632630e-08, ..., -8.618547e-02]]), 
3, 1.0, 'euclidean', None, 40, True, True, 8)


In [ ]:
tuning_result["silhouette"] = []
tuning_result["silhouette_error"] = []

tqdm_bar = tqdm(desc="Computing Silhouette score", total=len(model_list))
for i, _ in enumerate(model_list):
    try:
        silhouette = metrics.silhouette_score(
            X, label_list[i], sample_size=10000, random_state=SEED, n_jobs=-1
        )
        tuning_result["silhouette_error"].append("None")
    except Exception as e:
        print(e)
        silhouette = -1.1
        tuning_result["silhouette_error"].append(e)
    tuning_result["silhouette"].append(silhouette)
    tqdm_bar.update(1)
tqdm_bar.close()

In [ ]:
tuning_result["calinski_harabasz"] = []
tuning_result["calinski_harabasz_error"] = []

tqdm_bar = tqdm(desc="Computing Calinski Harabasz score", total=len(model_list))
for i, _ in enumerate(model_list):
    try:
        calinski_harabasz = metrics.calinski_harabasz_score(X, label_list[i])
        tuning_result["calinski_harabasz_error"].append("None")
    except Exception as e:
        print(e)
        calinski_harabasz = -1.1
        tuning_result["calinski_harabasz_error"].append(e)
    tuning_result["calinski_harabasz"].append(calinski_harabasz)
    tqdm_bar.update(1)
tqdm_bar.close()

In [ ]:
tuning_result["davies_bouldin"] = []
tuning_result["davies_bouldin_error"] = []

tqdm_bar = tqdm(desc="Computing Davies Bouldin score", total=len(model_list))
for i, _ in enumerate(model_list):
    try:
        davies_bouldin = metrics.davies_bouldin_score(X, label_list[i])
        tuning_result["davies_bouldin_error"].append("None")
    except Exception as e:
        print(e)
        davies_bouldin = -1.1
        tuning_result["davies_bouldin_error"].append(e)
    tuning_result["davies_bouldin"].append(davies_bouldin)
    tqdm_bar.update(1)
tqdm_bar.close()

In [ ]:
tuning_result["silhouette_corr"] = []
tuning_result["silhouette_corr_error"] = []

tqdm_bar = tqdm(desc="Computing Silhouette (correlation) score", total=len(model_list))
for i, _ in enumerate(model_list):
    try:
        silhouette = metrics.silhouette_score(
            X,
            label_list[i],
            sample_size=10000,
            random_state=SEED,
            n_jobs=-1,
            metric="correlation",
        )
        tuning_result["silhouette_corr_error"].append("None")
    except Exception as e:
        print(e)
        silhouette = -1.1
        tuning_result["silhouette_corr_error"].append(e)
    tuning_result["silhouette_corr"].append(silhouette)
    tqdm_bar.update(1)
tqdm_bar.close()

In [ ]:
# Display tuning results
tuning_result_df = pd.DataFrame(tuning_result)
tuning_result_df

In [ ]:
# Save tuning results
tuning_result_df.to_csv(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_tuning_result.csv", index=False
)

In [ ]:
# Combine model sets
tuple_objects = (
    model_list,
    label_list,
    outlier_list,
    exemplar_list,
    tuning_result,
)
len(tuple_objects), len(tuple_objects[0])

In [ ]:
# Save tuple of model sets
pickle.dump(
    tuple_objects, open(RESULT_DIR + "hdbscan/" + "skill_hdbscan_model_tuning_list.pkl", "wb")
)

## Evaluate cluster performance

In [ ]:
# Calculate other validation indices
model_validation = {
    "min_samples": tuning_result["min_samples"],
    "min_cluster_size": tuning_result["min_cluster_size"],
    "n_clusters": tuning_result["n_clusters"],
    "relative_validity": [],
    "cluster_persistence": [],
}

for i in tqdm(range(len(model_list))):
    model = model_list[i]

    # Retrieve relative validity
    """
    float
    A fast approximation of the Density Based Cluster Validity (DBCV) score. 
    The only difference, and the speed, comes from the fact that 
    this relative_validity_ is computed using 
    the mutual- reachability minimum spanning tree, i.e. minimum_spanning_tree_, 
    instead of the all-points minimum spanning tree used in the reference. 
    This score might not be an objective measure of 
    the goodness of clusterering. 
    It may only be used to compare results across 
    different choices of hyper-parameters, therefore is only a relative score.
    """
    relative_validity = model.relative_validity_

    # Retrieve cluster persistence
    """
    ndarray, shape (n_clusters, )
    A score of how persistent each cluster is. 
    A score of 1.0 represents a perfectly stable cluster 
    that persists over all distance scales, 
    while a score of 0.0 represents a perfectly ephemeral cluster. 
    These scores can be gauge the relative coherence 
    of the clusters output by the algorithm.
    """
    cluster_persistence = model.cluster_persistence_

    model_validation["relative_validity"].append(relative_validity)
    model_validation["cluster_persistence"].append(cluster_persistence)

In [ ]:
model_validation_df = pd.DataFrame(
    {
        "min_samples": model_validation["min_samples"],
        "min_cluster_size": model_validation["min_cluster_size"],
        "n_clusters": model_validation["n_clusters"],
        "relative_validity": model_validation["relative_validity"],
    }
)

In [ ]:
# Save validation results
model_validation_df.to_csv(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_validation_result.csv", index=False
)

In [ ]:
# Combine model sets
tuple_objects2 = (
    model_list,
    label_list,
    outlier_list,
    exemplar_list,
    tuning_result,
    model_validation,
)
len(tuple_objects2), len(tuple_objects2[0])

In [ ]:
# Save tuple
pickle.dump(
    tuple_objects2,
    open(OUTPUT_DIR + "hdbscan/" + "skill_hdbscan_tuning_validation_list.pkl", "wb"),
)

#### Number of clusters found

In [ ]:
# List of markers for subplots
marker_dict = {
    ".": "point",
    ",": "pixel",
    "o": "circle",
    "v": "triangle_down",
    "^": "triangle_up",
    "<": "triangle_left",
    ">": "triangle_right",
    "1": "tri_down",
    "2": "tri_up",
    "3": "tri_left",
    "4": "tri_right",
    "8": "octagon",
    "s": "square",
    "p": "pentagon",
    "*": "star",
    "h": "hexagon1",
    "H": "hexagon2",
    "+": "plus",
    "x": "x",
    "D": "diamond",
    "d": "thin_diamond",
    "|": "vline",
    "_": "hline",
    "P": "plus_filled",
    "X": "x_filled",
    0: "tickleft",
    1: "tickright",
    2: "tickup",
    3: "tickdown",
    4: "caretleft",
    5: "caretright",
    6: "caretup",
    7: "caretdown",
    8: "caretleftbase",
    9: "caretrightbase",
    10: "caretupbase",
    11: "caretdownbase",
}
marker_list = list(marker_dict.keys())

In [ ]:
palette = sns.color_palette("hls", n_colors=len(min_cluster_size_list))

# The higher the better
fig = plt.figure(figsize=(12, 6))
for idx, min_cluster_size in enumerate(min_cluster_size_list):
    plt.plot(
        tuning_result_df["min_samples"].loc[
            tuning_result_df["min_cluster_size"] == min_cluster_size
        ],        
        tuning_result_df["n_clusters"].loc[
            tuning_result_df["min_cluster_size"] == min_cluster_size
        ],        
        label="Min cluster size = "
        + str(min_cluster_size),
        color=palette[idx],
        marker=marker_list[idx]
    )
plt.ylabel("Number of clusters")
# plt.yticks(np.unique(model_result_df["number_clusters"]))
plt.xlabel("Min samples")
# plt.xticks(np.unique(model_result_df["min_samples"]))
plt.title("Number of clusters for HDBSCAN clustering")
plt.legend(loc="best")
plt.grid()

# Saving plot as image
fig.savefig(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_n_clusters_plot.png",
#     bbox_inches="tight",
#     dpi=150,
)

# Show plot
plt.show()

#### Silhoulette score

In [ ]:
palette = sns.color_palette("hls", n_colors=len(min_samples_list))

# The higher the better
fig, ax1 = plt.subplots(figsize=(12, 6))
for idx, min_samples in enumerate(min_samples_list):
    ax1.plot(
        tuning_result_df["min_cluster_size"].loc[
            tuning_result_df["min_samples"] == min_samples
        ],
        tuning_result_df["silhouette"].loc[
            tuning_result_df["min_samples"] == min_samples
        ],
        label="Min samples = "
        + str(min_samples), color=palette[idx],
        marker=marker_list[idx]
    )
ax1.set_xlabel("Min Cluster Size")
# ax1.set_xticks(min_cluster_size_list)
ax1.set_ylabel("Silhouette Score")
ax1.set_title("Silhouette score for HDBSCAN clustering")
ax1.legend(loc="best")
ax1.grid()

# Saving plot as image
fig.savefig(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_silhouette_plot.png",
#     bbox_inches="tight",
#     dpi=150,
)

# Show plot
plt.show()

#### Calinski Harabasz Score

In [ ]:
palette = sns.color_palette("hls", n_colors=len(min_samples_list))

# The higher the better
fig, ax1 = plt.subplots(figsize=(12, 6))
for idx, min_samples in enumerate(min_samples_list):
    ax1.plot(
        tuning_result_df["min_cluster_size"].loc[
            tuning_result_df["min_samples"] == min_samples
        ],
        tuning_result_df["calinski_harabasz"].loc[
            tuning_result_df["min_samples"] == min_samples
        ],
        label="Min samples = "
        + str(min_samples), color=palette[idx],
        marker=marker_list[idx]
    )
ax1.set_xlabel("Min Cluster Size")
# ax1.set_xticks(min_cluster_size_list)
ax1.set_ylabel("Calinski Harabasz Score")
ax1.set_title("Calinski Harabasz score for HDBSCAN clustering")
ax1.legend(loc="best")
ax1.grid()

# Saving plot as image
fig.savefig(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_calinski_harabasz_plot.png",
#     bbox_inches="tight",
#     dpi=150,
)

# Show plot
plt.show()

#### Davies-Bouldin Score

In [ ]:
palette = sns.color_palette("hls", n_colors=len(min_samples_list))

# Closer to 0 is better
fig, ax1 = plt.subplots(figsize=(12, 6))
for idx, min_samples in enumerate(min_samples_list):
    ax1.plot(
        tuning_result_df["min_cluster_size"].loc[
            tuning_result_df["min_samples"] == min_samples
        ],
        tuning_result_df["davies_bouldin"].loc[
            tuning_result_df["min_samples"] == min_samples
        ],
        label="Min samples = "
        + str(min_samples), color=palette[idx],
        marker=marker_list[idx]
    )
ax1.set_xlabel("Min Cluster Size")
# ax1.set_xticks(min_cluster_size_list)
ax1.set_ylabel("Davies-Bouldin Score")
ax1.set_title("Davies-Bouldin score for HDBSCAN clustering")
ax1.legend(loc="best")
ax1.grid()

# Saving plot as image
fig.savefig(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_davies_bouldin_plot.png",
#     bbox_inches="tight",
#     dpi=150,
)

# Show plot
plt.show()

#### Relative validity

In [ ]:
palette = sns.color_palette("hls", n_colors=len(min_cluster_size_list))

# The higher the better
plt.figure(figsize=(12, 6))
for idx, min_cluster_size in enumerate(min_cluster_size_list):
    plt.plot(
        model_validation_df["min_samples"].loc[
            model_validation_df["min_cluster_size"] == min_cluster_size
        ],
        model_validation_df["relative_validity"].loc[
            model_validation_df["min_cluster_size"] == min_cluster_size
        ],
        label="Min cluster size = " + str(min_cluster_size),
        color=palette[idx],
        marker=marker_list[idx]
    )
plt.ylabel("Relative validity")
plt.xlabel("Min samples")
plt.title("Relative validity for HDBSCAN clustering")
plt.legend(loc="best")
plt.grid()

# Saving plot as image
fig.savefig(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_relative-validity_plot.png",
#     bbox_inches="tight",
#     dpi=150,
)

# Show plot
plt.show()

In [ ]:
# best_index = np.argmax(model_validation['relative_validity'])
best_index = int(
    np.where(
        (np.array(tuning_result["min_samples"]) == 5)
        & (np.array(tuning_result["min_cluster_size"]) == 50)
    )[0]
)
print(best_index)

best_min_samples = tuning_result_df["min_samples"][best_index]
best_min_cluster_size = tuning_result_df["min_cluster_size"][best_index]
print(best_min_samples, best_min_cluster_size)
best_labels = label_list[best_index]
best_n_clusters = tuning_result_df["n_clusters"][best_index]
best_n_noises = tuning_result_df["n_noises"][best_index]
print(best_n_clusters, best_n_noises)

In [ ]:
pca_datapoint = np.load(OUTPUT_DIR + "visualization/" + "skill_halfsize_pca_datapoints.npy")
umap_datapoint = np.load(OUTPUT_DIR + "visualization/" + "skill_halfsize_umap_datapoints.npy")
tsne_datapoint = np.load(OUTPUT_DIR + "visualization/" + "skill_halfsize_tsne_datapoints.npy")

In [ ]:
plot_title = (
    "HDBSCAN cluster visualization on skill embeddings \n Number of clusters = "
    + str(best_n_clusters) + " - Number of noises = " + str(best_n_noises)
    + "\n Min samples = "
    + str(best_min_samples)
    + " - Min cluster size = "
    + str(best_min_cluster_size)
)
plot_filename = (
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_optimal-cluster-visualization.png"
)
palette = sns.color_palette("nipy_spectral", as_cmap=True)
# colors = cm.nipy_spectral(best_labels.astype(float) / best_k)
colors = best_labels

# Visualize clusters with PCA, UMAP, and t-SNE
visualize_cluster(
    plot_title,
    (12, 15),
    colors,
    palette,
    4,
    pca_datapoint,
    tsne_datapoint,
    umap_datapoint,
    pca=True,
    tsne=True,
    compute_umap=True,
    save_plots=True,
    plot_file=plot_filename,
)

## Display top words of each cluster

In [ ]:
df_clusters = skill_docs.copy(deep=True)
df_clusters["model_type"] = "genie"
df_clusters["skill_n_clusters"] = best_n_clusters
df_clusters['skill_cluster_label'] = best_labels
df_clusters

In [ ]:
wordcount = {}
sorted_wordcount = {}
for i in range(best_n_clusters):
    skills = df_clusters[df_clusters["skill_cluster_label"] == i]["skill"].values
    skills = " ".join(" ".join(skills).split())
    wordcount[i] = {}
    for j in skills.split():
        if j in wordcount[i]:
            wordcount[i][j] += 1
        else:
            wordcount[i][j] = 1
    sorted_wordcount[i] = sorted(wordcount[i].items(), key=lambda x: x[1], reverse=True)

In [ ]:
# tmp = df_clusters.copy(deep=True)
# tmp = tmp.loc[tmp['skill_cluster_label'] == 24]
# tmp.iloc[50:100, ]

In [ ]:
topwords = {}
for key, i in sorted_wordcount.items():
    print("Cluster " + str(key) + ": ", end="")
    topwords[key] = ""
    n = 0
    for newkey, j in sorted_wordcount[key][:10]:
        print(newkey + "|", end="")
        topwords[key] = topwords[key] + newkey + "|"
        if n == 10:
            print("\n------------ ", end="")
        n += 1
    print()

In [ ]:
topwords2 = {}
for key, i in sorted_wordcount.items():
    print("Cluster "+str(key)+": ", end='')
    topwords2[key] = ''
    for newkey, j in sorted_wordcount[key][10:20]:
        print(newkey + '|', end='')
        topwords2[key] = topwords2[key] + newkey + '|'
    print()

In [ ]:
topwords3 = {}
for key, i in sorted_wordcount.items():
    print("Cluster "+str(key)+": ", end='')
    topwords3[key] = ''
    for newkey, j in sorted_wordcount[key][20:30]:
        print(newkey + '|', end='')
        topwords3[key] = topwords3[key] + newkey + '|'
    print()

## Save cluster results to file

In [ ]:
df_clusters.to_csv(
    RESULT_DIR + "hdbscan/" + "skill_hdbscan_optimal-cluster-labels.csv", index=False
)

In [ ]:
# Save tuple of model sets
pickle.dump(
    (sorted_wordcount, topwords, topwords2, topwords3),
    open(RESULT_DIR + "hdbscan/" + "skill_hdbscan_optimal-topwords.pkl", "wb"),
)